## List all training scans

In [92]:
import os
import multiprocessing
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
# from tensorboardX import SummaryWriter

import utils

In [40]:
with open('/home/hoyeung/blob_matterport3d/semantics/asknav_tr_room_types.txt', "w") as f:
    for room_type in tr_room_types:
        f.write("{}\n".format(room_type))

In [127]:
with open('/home/hoyeung/blob_matterport3d/semantics/asknav_tr_room_types.txt', "r") as f:
    room_types = f.read().split('\n')
room_types[:-1]

['-',
 'B',
 'C',
 'S',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [43]:
len(room_types[:-1])

30

In [44]:
len(tr_room_types)

30

In [23]:
os.environ['PT_DATA_DIR'] ='/home/hoyeung/blob_matterport3d/'

# 1 sim 1 scan
# 56s for 56 scans
all_tr_scans = ['ac26ZMwG7aT',
                'VzqfbhrpDEA',
                'vyrNrziPKCB',
                'mJXqzFtmKg4',
                'Vvot9Ly1tCj',
                'gTV8FGcVJC9',
                '1pXnuDYAj8r',
                'cV4RVeZvu5T',
                'p5wJjkQkbXX',
                '2n8kARJN3HM',
                'EDJbREhghzL',
                '8WUmhLawc2A',
                'SN83YJsR3w2',
                'JmbYfDe2QKZ',
                '759xd9YjKW5',
                'uNb9QFRL6hY',
                'JeFG25nYj2p',
                'qoiz87JEwZ2',
                'PX4nDJXEHrG',
                'D7N2EKCX4Sj',
                'aayBHfsNo7d',
                'b8cTxDM8gDG',
                '82sE5b5pLXE',
                '5LpN3gDmAk7',
                '1LXtFkjw3qL',
                'kEZ7cmS4wCh',
                'ULsKaCPVFJR',
                'ZMojNkEp431',
                'e9zR4mvMWw7',
                'VFuaQ6m2Qom',
                'rPc6DW4iMge',
                'S9hNv5qa7GM',
                '5q7pvUzZiYa',
                'pRbA3pwrgk9',
                'E9uDoFAP3SH',
                '17DRP5sb8fy',
                'XcA2TqTSSAj',
                'i5noydFURQK',
                '7y3sRwLe3Va',
                'sT4fr6TAbpF',
                'r47D5H71a5s',
                'jh4fc5c5qoQ',
                'r1Q1Z4BcV1o',
                'Uxmj2M2itWa',
                'VVfe2KiqLaN',
                'ur6pFq6Qu1A',
                'B6ByNegPMKs',
                'JF19kD82Mey',
                'sKLMLpTHeUy',
                '29hnd4uzFmX',
                'HxpKQynjfin',
                'V2XKFyX4ASd',
                'VLzqgDo317F',
                's8pcmisQ38h',
                'GdvgFV5R1Z5',
                'Pm6F8kyY3z2']

# from training environments
tr_room_types = ['-',
                 'B',
                 'C',
                 'S',
                 'Z',
                 'a',
                 'b',
                 'c',
                 'd',
                 'e',
                 'f',
                 'g',
                 'h',
                 'i',
                 'j',
                 'k',
                 'l',
                 'm',
                 'n',
                 'o',
                 'p',
                 'r',
                 's',
                 't',
                 'u',
                 'v',
                 'w',
                 'x',
                 'y',
                 'z']

## Image features

In [54]:
img_features_path = "img_features/ResNet-152-imagenet.tsv"
image_h, image_w, vfov, all_img_features = utils.load_img_features(os.path.join(os.getenv('PT_DATA_DIR'), hparams['img_features_path']))

Loading image features from /home/hoyeung/blob_matterport3d/img_features/ResNet-152-imagenet.tsv


## Dataset class

In [55]:
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

class PanoramaDataset(Dataset):
    """Indoor panoramic view dataset"""
    
    def __init__(self, image_features):
        
        # load scans
        self.image_features = image_features
    
    def __len__(self):
        return len(self.image_features)
    
    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample = {'f':self.image_features[idx]}
        
        return sample

In [56]:
dum1 = PanoramaDataset(all_img_features)
dum2 = PanoramaDataset(all_img_features)

In [59]:
dum1.image_features is dum2.image_features

True

In [ ]:
tr_panorama_dataset = PanoramaDataset(scans=all_tr_scans,
                                      tr_room_types=tr_room_types)

In [24]:
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

class PanoramaDataset(Dataset):
    """Indoor panoramic view dataset"""
    
    def __init__(self, rm_labels, feature_ids, tr_room_types, transform=None):
        
        # load scans
        self.room_labels = rm_labels
        self.feature_ids = feature_ids
        self.room_types = tr_room_types
        self.transform = transform
    
    def __len__(self):
        return len(self.feature_ids)
    
    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        feature_id = self.feature_ids[idx]
        feature = all_img_features[feature_id[0]][feature_id[1]]
        room_idx = self.room_types.index(self.room_labels[idx])
        sample = {'feature': torch.from_numpy(feature), 'room': torch.tensor(room_idx)}
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample

In [47]:
import torch.nn as nn
hidden_layers = [1024, 512]

fc_layers = []
fc_in_size = 2048
for fc_out_size in hidden_layers:
    fc_layer = nn.Linear(fc_in_size, fc_out_size)
    fc_layers.append(fc_layer)
    fc_in_size = fc_out_size
final_layer = nn.Linear(fc_out_size, 30)

In [48]:
fc_layers

[Linear(in_features=2048, out_features=1024, bias=True),
 Linear(in_features=1024, out_features=512, bias=True)]

In [49]:
final_layer

Linear(in_features=512, out_features=30, bias=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(2048, 30)
#         self.fc1 = nn.Linear(2048, 30)
#         self.fc2 = nn.Linear(1024, 512)
#         self.fc3 = nn.Linear(512, 30)
        
    def forward(self, x):
#         print (x.shape)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        x = self.fc(x)
        return x

In [64]:
np.random.choice(all_tr_scans, size=2, replace=False)

array(['SN83YJsR3w2', 'b8cTxDM8gDG'], dtype='<U11')

In [26]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

def retrieve_rm_labels_and_feature_ids(scans):
    rm_labels = []
    feature_ids = []
    for scan in scans:
        scan_G = utils.load_nav_graphs(scan)
        scan_panos_to_region = utils.load_panos_to_region(scan,"")
        for n in scan_G.nodes:
            room_label = scan_panos_to_region[n]
            long_id = scan + '_' + n
            for viewix in range(36):
                rm_labels.append(rm_labels)
                feature_ids.append((long_id, viewix))
    return rm_labels, feature_ids
    
# take training scans, return training seen, val seen and val unseen indices
def train_val(scans, folds, hparams):
    
    if len(all_tr_scans) % folds != 0:
        raise ValueError('len(all_tr_scans) must be dividable by folds.')
    
    # shuffle data and split into folds
    all_tr_scans_shuffle = all_tr_scans.copy()
    random.shuffle(all_tr_scans_shuffle)
    all_tr_scans_chunks = list(chunks(all_tr_scans_shuffle, folds))
    
    # Loss critierion
    criterion = nn.CrossEntropyLoss()
    
    # Loop through folds
    for unseen_split_chunk_idx in range(len(all_tr_scans_chunks)):
        classifier_seen_scans = []
        classifier_unseen_scans = []
        for i in range(len(all_tr_scans_chunks)):
            if i == unseen_split_chunk_idx:
                classifier_unseen_scans += all_tr_scans_chunks[i]
            else:
                classifier_seen_scans += all_tr_scans_chunks[i]
    
        # feature ids and labels
        seen_rm_labels, seen_feature_ids = retrieve_rm_labels_and_feature_ids(classifier_seen_scans)
        val_unseen_rm_labels, val_unseen_feature_ids = retrieve_rm_labels_and_feature_ids(classifier_unseen_scans)
        
        # split seen indices between train and val seen
        tr_seen_rm_labels = []
        tr_seen_feature_ids = []
        val_seen_rm_labels = []
        val_seen_feature_ids = []
        val_seen_idx = np.random.choice(len(seen_feature_ids), size=len(val_unseen_feature_ids), replace=False)
        for i in range(len(seen_feature_ids)):
            if i in val_seen_idx:
                val_seen_rm_labels.append(seen_rm_labels[i])
                val_seen_feature_ids.append(seen_feature_ids[i])
            else:
                tr_seen_rm_labels.append(seen_rm_labels[i])
                tr_seen_feature_ids.append(seen_feature_ids[i])
            
        # make Dataset objects
        tr_dataset = PanoramaDataset(tr_seen_rm_labels, tr_seen_feature_ids, tr_room_types)
        val_seen_dataset = PanoramaDataset(val_seen_rm_labels, val_seen_feature_ids, tr_room_types)
        val_unseen_dataset = PanoramaDataset(val_unseen_rm_labels, val_unseen_feature_ids, tr_room_types)
        
        return train(tr_dataset, val_seen_dataset, val_unseen_dataset, hparams)
    
def train(tr_dataset, val_seen_dataset, val_unseen_dataset, n_iters, save_every, device):
    
    # make DataLoader
    tr_data_loader = DataLoader(tr_dataset, batch_size=200, shuffle=True, num_workers=multiprocessing.cpu_count())
    val_seen_data_loader = DataLoader(val_seen_dataset, batch_size=500, shuffle=False, num_workers=multiprocessing.cpu_count())
    val_unseen_data_loader = DataLoader(val_unseen_dataset, batch_size=500, shuffle=False, num_workers=multiprocessing.cpu_count())

    # make net
    net = Net()
    net.to(device)
    
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    
#     SW = SummaryWriter(tensorboard_dir, flush_secs=30)
    

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
len(all_tr_scans) // 7

8

In [20]:
train(scans=all_tr_scans, folds=7)

> <ipython-input-18-652cb6618a2d>(32)train()
-> for unseen_split_chunk_idx in range(len(all_tr_scans_chunks)):
(Pdb) tr_seen_feature_ids[:10]
[('p5wJjkQkbXX_182ecddefe384e829ff8bae132f93198', 0), ('p5wJjkQkbXX_182ecddefe384e829ff8bae132f93198', 1), ('p5wJjkQkbXX_182ecddefe384e829ff8bae132f93198', 2), ('p5wJjkQkbXX_182ecddefe384e829ff8bae132f93198', 3), ('p5wJjkQkbXX_182ecddefe384e829ff8bae132f93198', 4), ('p5wJjkQkbXX_182ecddefe384e829ff8bae132f93198', 5), ('p5wJjkQkbXX_182ecddefe384e829ff8bae132f93198', 6), ('p5wJjkQkbXX_182ecddefe384e829ff8bae132f93198', 8), ('p5wJjkQkbXX_182ecddefe384e829ff8bae132f93198', 9), ('p5wJjkQkbXX_182ecddefe384e829ff8bae132f93198', 10)]
(Pdb) tr_seen_feature_ids[5]
('p5wJjkQkbXX_182ecddefe384e829ff8bae132f93198', 5)
(Pdb) len(tr_seen_feature_ids)
225180
(Pdb) len(val_unseen_rm_labels_
*** SyntaxError: unexpected EOF while parsing
(Pdb) len(val_unseen_rm_labels)
18108
(Pdb) len(val_seen_rm_labels)
18108
(Pdb) set(val_seen_rm_labels).symmetric_difference(set(

(Pdb) set(val_unseen_feature_ids) & set(val_seen_feature_ids)
set()
(Pdb) set(tr_sseen_feature_ids) & set(val_seen_feature_ids)
*** NameError: name 'tr_sseen_feature_ids' is not defined
(Pdb) set(tr_seen_feature_ids) & set(val_seen_feature_ids)
set()
(Pdb) q


BdbQuit: 

## Dataset class

In [50]:
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

class PanoramaDataset(Dataset):
    """Indoor panoramic view dataset"""
    
    def __init__(self, scans, tr_room_types, transform=None):
        
        # load scans
        self.scans = scans
        self.room_labels, self.feature_ids = self._load_labels_and_ids()
        
        self.room_types = tr_room_types
        
        self.transform = transform
        
    def _load_labels_and_ids(self):
        room_labels = []
        feature_ids = []
        for scan in self.scans:
            scan_G = utils.load_nav_graphs(scan)
            scan_panos_to_region = utils.load_panos_to_region(scan,"")
            for n in scan_G.nodes:
                room_label = scan_panos_to_region[n]
                long_id = scan + '_' + n
                for viewix in range(36):
                    room_labels.append(room_label)
                    feature_ids.append((long_id, viewix))
        return room_labels, feature_ids
    
    def __len__(self):
        return len(self.room_labels)
    
    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        feature_id = self.feature_ids[idx]
        feature = all_img_features[feature_id[0]][feature_id[1]]
        room_idx = self.room_types.index(self.room_labels[idx])
        sample = {'feature': torch.from_numpy(feature), 'room': torch.tensor(room_idx)}
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample

In [8]:
torch.tensor(5)

tensor(5)

In [62]:
tr_panorama_dataset = PanoramaDataset(scans=all_tr_scans,
                                      tr_room_types=tr_room_types)

In [52]:
tr_panorama_dataset[100]

{'feature': tensor([0.3763, 0.0723, 0.3632,  ..., 0.0914, 0.0377, 0.3861]),
 'room': tensor(15)}

In [19]:
tr_panorama_dataset.room_types[15]

'k'

## DataLoader

In [63]:
tr_data_loader = DataLoader(tr_panorama_dataset, batch_size=200, shuffle=True, num_workers=multiprocessing.cpu_count())

In [ ]:
for i, sampled_batch in enumerate(tr_data_loader):
    print (i, sampled_batch['feature'].shape, sampled_batch['room'].shape)

## Classifier model and optim code

Net(
  (fc): Linear(in_features=2048, out_features=30, bias=True)
)

In [65]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Training

In [ ]:
for epoch in range(10000):
    
    running_loss = 0.0
    running_losses = []
    for i, data in enumerate(tr_data_loader, 0):
        # shape (batch size, 2048)
        features = data['feature'].to(device)
        # shape (batch_size,)
        rooms = data['room'].to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward pass
        pred = net(features)
        loss = criterion(pred, rooms)
        loss.backward()
        optimizer.step()
        
        # print stats
        running_loss += loss.item()
        if i % 500 == 499:
            running_loss = 0.0
            running_losses.append(running_loss)
            
print("done")

In [2]:
d = {'a':1, 'b':2}
d.items()

dict_items([('a', 1), ('b', 2)])

In [64]:
from collections import defaultdict

In [66]:
defaultdict(float)

defaultdict(float, {})

In [67]:
float()

0.0

In [68]:
for a,b,c in zip([1,2,3], [4,5,6], [7,8,9]):
    print (a,b,c)

1 4 7
2 5 8
3 6 9


# Debug

In [3]:
import os
import time
import sys
import multiprocessing
import random
import numpy as np

from pprint import pprint
import argparse
from argparse import Namespace
import json
from collections import defaultdict

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
# from tensorboardX import SummaryWriter

from model import FFSemanticClassifier
from dataset import PanoramaDataset
import utils
from utils import timeSince
from flags_semantics import make_parser

In [5]:
DATA_DIR = '/home/hoyeung/blob_matterport3d/'
scans_path = os.path.join(DATA_DIR, 'semantics/asknav_tr_scans.txt')
room_types_path = os.path.join(DATA_DIR, 'semantics/asknav_tr_room_types.txt')

In [6]:
# Define scans
with open(scans_path, "r") as fh:
    scans = fh.read().split("\n")
    scans = scans[:-1]

# Define room types
with open(room_types_path, "r") as fh:
    room_types = fh.read().split("\n")
    room_types = room_types[:-1]

In [29]:
def read_rm_labels_and_feature_ids(idx_save_path):
    feature_ids = []
    room_labels = []
    with open(idx_save_path, 'r') as fh:
        lines = fh.read().split('\n')
        lines = lines[:-1]
        for line in lines:
            long_id, viewix, room_label = line.split('\t')
            room_labels.append(room_label)
            feature_ids.append((long_id, int(viewix)))
            
    return feature_ids, room_labels

In [17]:
img_features_path = "img_features/ResNet-152-imagenet.tsv"
image_h, image_w, vfov, all_img_features = utils.load_img_features(os.path.join(DATA_DIR, img_features_path))

Loading image features from /home/hoyeung/blob_matterport3d/img_features/ResNet-152-imagenet.tsv


In [9]:
len(room_types)

30

In [27]:
exp_dir = '/home/hoyeung/blob_experiments/output_local/20200201_debug_semantics/debug_pre_tunning_1'
tr_idx_save_path = os.path.join(exp_dir, 'train_indices.txt')
val_seen_idx_save_path = os.path.join(exp_dir, 'val_seen_indices.txt')
val_unseen_idx_save_path = os.path.join(exp_dir, 'val_unseen_indices.txt')

In [30]:
tr_seen_feature_ids, tr_seen_rm_labels = read_rm_labels_and_feature_ids(tr_idx_save_path)
val_seen_feature_ids, val_seen_rm_labels = read_rm_labels_and_feature_ids(val_seen_idx_save_path)
val_unseen_feature_ids, val_unseen_rm_labels = read_rm_labels_and_feature_ids(val_unseen_idx_save_path)

ValueError: not enough values to unpack (expected 3, got 2)

In [24]:
tr_seen_feature_ids[0].split()

"('5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf', 0)"

# Baselines

In [2]:
import json
import os
from collections import Counter

In [17]:
DATA_DIR = '/home/hoyeung/blob_matterport3d/'
room_types_path = os.path.join(DATA_DIR, "semantics/asknav_tr_room_types.txt")
with open(room_types_path, "r") as fh:
    room_types = fh.read().split("\n")
    room_types = room_types[:-1]

In [113]:
exp_dir = '/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/classifier_seed_42_batch_size_100_lr_0.001_layers_2/'
with open(os.path.join(exp_dir,'train_indices.txt'), 'r') as fh:
    data = fh.read().split('\n')
    
len(data)

224317

In [111]:
exp_dir = '/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/classifier_seed_42_batch_size_100_lr_0.001_layers_2/'
with open(os.path.join(exp_dir,'val_seen_indices.txt'), 'r') as fh:
    data = fh.read().split('\n')
    
len(data)

18541

In [114]:
exp_dir = '/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/classifier_seed_42_batch_size_100_lr_0.001_layers_2/'
with open(os.path.join(exp_dir,'val_unseen_indices.txt'), 'r') as fh:
    data = fh.read().split('\n')
    
len(data)

18541

In [8]:
exp_dir = '/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/classifier_seed_42_batch_size_100_lr_0.001_layers_2/'
with open(os.path.join(exp_dir,'val_seen_results.json'), 'r') as fh:
    data = json.load(fh)

len(data)

4248

In [4]:
exp_dir = '/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/classifier_seed_42_batch_size_100_lr_0.001_layers_2/'with open(os.path.join(exp_dir,'val_unseen_results.json'), 'r') as fh:
    data = json.load(fh)

In [115]:
exp_dir = '/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/classifier_seed_42_batch_size_100_lr_0.001_layers_2/'
feature_ids = []
room_labels = []

with open(os.path.join(exp_dir,'train_indices.txt'), 'r') as fh:
    lines = fh.read().split('\n')[:-1]
    for line in lines:
        long_id, viewix, room_label = line.split('\t')
        room_labels.append(int(room_label))
        feature_ids.append((long_id, int(viewix)))

In [117]:
exp_dir = '/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/classifier_seed_42_batch_size_100_lr_0.001_layers_2/'
feature_ids = []
room_labels = []

with open(os.path.join(exp_dir,'val_seen_indices.txt'), 'r') as fh:
    lines = fh.read().split('\n')[:-1]
    for line in lines:
        long_id, viewix, room_label = line.split('\t')
        room_labels.append(int(room_label))
        feature_ids.append((long_id, int(viewix)))

In [119]:
exp_dir = '/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/classifier_seed_42_batch_size_100_lr_0.001_layers_2/'
feature_ids = []
room_labels = []

with open(os.path.join(exp_dir,'val_unseen_indices.txt'), 'r') as fh:
    lines = fh.read().split('\n')[:-1]
    for line in lines:
        long_id, viewix, room_label = line.split('\t')
        room_labels.append(int(room_label))
        feature_ids.append((long_id, int(viewix)))

In [25]:
room_types.index('l')

16

In [29]:
room_types[1]

'B'

In [113]:
exp_dir = '/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/classifier_seed_42_batch_size_100_lr_0.001_layers_2/'
with open(os.path.join(exp_dir,'train_indices.txt'), 'r') as fh:
    data = fh.read().split('\n')
    
len(data)

224317

In [116]:
# training
Counter(room_labels).most_common()

[(12, 36984),
 (6, 30134),
 (5, 19586),
 (20, 14215),
 (15, 12696),
 (16, 11687),
 (22, 10581),
 (0, 10326),
 (8, 9678),
 (9, 8933),
 (10, 8532),
 (17, 6524),
 (19, 6520),
 (29, 5865),
 (28, 5605),
 (18, 4538),
 (7, 3832),
 (21, 3276),
 (3, 2920),
 (25, 2726),
 (13, 2118),
 (11, 1761),
 (26, 1554),
 (14, 1500),
 (23, 819),
 (24, 652),
 (2, 591),
 (27, 67),
 (1, 65),
 (4, 31)]

In [118]:
# val seen
Counter(room_labels).most_common()

[(12, 3120),
 (6, 2446),
 (5, 1690),
 (20, 1157),
 (15, 1092),
 (16, 985),
 (22, 867),
 (0, 798),
 (8, 798),
 (9, 679),
 (10, 648),
 (17, 568),
 (19, 500),
 (28, 479),
 (29, 471),
 (18, 358),
 (7, 308),
 (21, 288),
 (3, 248),
 (25, 226),
 (13, 186),
 (11, 147),
 (26, 138),
 (14, 120),
 (23, 81),
 (24, 68),
 (2, 57),
 (1, 7),
 (4, 5),
 (27, 5)]

In [120]:
# val unseen
Counter(room_labels).most_common()

[(12, 3168),
 (6, 2736),
 (22, 1656),
 (18, 1368),
 (5, 1260),
 (28, 1260),
 (10, 1080),
 (20, 1044),
 (0, 900),
 (8, 792),
 (9, 792),
 (15, 792),
 (16, 612),
 (24, 576),
 (19, 324),
 (14, 72),
 (29, 72),
 (7, 36)]

In [136]:
room_types[15]

'k'

In [123]:
print ('val seen baselines:')
print('Top 1 acc = {}'.format(3120/18541.0))
print('Top 3 acc = {}'.format((3120 + 2446 + 1690)/18541.0))
print('Top 5 acc = {}'.format((3120 + 2446 + 1690 + 1157 + 1092)/18541.0))

val seen baselines:
Top 1 acc = 0.16827571328407315
Top 3 acc = 0.391348902432447
Top 5 acc = 0.5126476457580498


In [124]:
print ('val seen baselines:')
print('Top 1 acc = {}'.format(3168/18541.0))
print('Top 3 acc = {}'.format((3168 + 2736 + 1656)/18541.0))
print('Top 5 acc = {}'.format((3168 + 2736 + 1656 + 1368 + 1260)/18541.0))

val seen baselines:
Top 1 acc = 0.17086457041152042
Top 3 acc = 0.40774499757294647
Top 5 acc = 0.5494849253006849


In [50]:
set(data) == set(data2)

True

In [48]:
with open("/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/semantic_classifier_debug_3/val_unseen_indices.txt", 'r') as f:
    data2 = f.read().split('\n')
data2

['rPc6DW4iMge_c688725841444971ab4788b382c60643\t0\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t1\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t2\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t3\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t4\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t5\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t6\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t7\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t8\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t9\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t10\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t11\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t12\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t13\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t14\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t15\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t16\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c

In [49]:
with open("/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/semantic_classifier_debug_2/val_unseen_indices.txt", 'r') as f:
    data = f.read().split('\n')
data

['rPc6DW4iMge_c688725841444971ab4788b382c60643\t0\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t1\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t2\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t3\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t4\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t5\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t6\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t7\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t8\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t9\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t10\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t11\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t12\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t13\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t14\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t15\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c60643\t16\t22',
 'rPc6DW4iMge_c688725841444971ab4788b382c

# Make next room prediction dataset

In [ ]:
def _retrieve_curr_viewix_and_next_vertex(self):
    '''
    Returns:
        viewix_batch: list of integers, length = batch_size.
        closest_vertex_batch: list of viewpointId strings, length = batch_size.
    '''
    # time keeping
    time_report = defaultdict(int)

    next_vertex_batch = [''] * self.batch_size

    start_time = time.time()
    # list len=batch_size, each int
    # list len=batch_size, each a dictionary
    viewix_batch, closest_navigable_locations_batch = self._getStates()
    time_report['explore_get_states'] += time.time() - start_time

    start_time = time.time()
    # check if whether we are looking at the closest navigable vertex directly.
    for i, closest_vertex in enumerate(closest_navigable_locations_batch):
        # only a few rotation angles are connected to another vertex
        if closest_vertex:
            # 1) directly facing -- within 30deg of center of view
            # 2,3) already looking up/down, but vertex is more than 15 deg further up/down
            if 	self._check_center(closest_vertex) or \
                (viewix_batch[i] // 12 == 2 and self._check_above(closest_vertex)) or \
                (viewix_batch[i] // 12 == 0 and self._check_below(closest_vertex)):
                next_vertex_batch[i] = closest_vertex.viewpointId
            # otherwise, remain ''
    time_report['explore_find_direct_vertex'] += time.time() - start_time

    # list length batch_size, list length batch_size
    return viewix_batch, next_vertex_batch, time_report